In [1]:

import cv2
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os
from pathlib import Path
if Path('.').resolve().name != 'Arena':
    os.chdir('../Arena')
from analysis.pose import ArenaPose

In [2]:
ap = ArenaPose('front', 'deeplabcut')

In [5]:
exp_dir = '../output/experiments/PV80'
videos = list(Path(exp_dir).rglob('*.mp4')) + list(Path(exp_dir).rglob('*.avi'))
videos = [v for v in videos if 'front' in v.name and ap.get_predicted_cache_path(v).exists()]
videos = sorted(videos, key=lambda x: x.stem.split('_')[1])
videos

[PosixPath('../output/experiments/PV80/20221126/block1/videos/front_20221126T112536.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block2/videos/front_20221126T113356.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block4/videos/front_20221126T121008.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block5/videos/front_20221126T121826.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block6/videos/front_20221126T122642.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block8/videos/front_20221126T131828.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block9/videos/front_20221126T132646.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block10/videos/front_20221126T141010.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block11/videos/front_20221126T141828.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block12/videos/front_20221126T142701.mp4'),
 PosixPath('../output/experiments/PV80/20221126/block13/videos/front_20221126T151008.mp

In [6]:
ap.load_predicted_video(videos[-1])

left_ear                              nose                        \
           cam_x       cam_y      prob       cam_x       cam_y      prob   
0     804.053162  108.034637  0.849663  675.697937   70.124275  0.014824   
1     805.352356  107.168114  0.949083  777.429871  125.789452  0.014845   
2     807.108704  106.105415  0.922115  492.459900    2.250166  0.012374   
3     807.599792  106.435974  0.948555  776.382019  131.063736  0.013475   
4     807.219788  106.979240  0.959419  776.151062  130.885513  0.022431   
...          ...         ...       ...         ...         ...       ...   
3385  583.906372    2.653002  0.108534  490.684082    4.375213  0.307732   
3386  583.914917    2.653224  0.108414  490.687622    4.371529  0.307740   
3387  583.917725    2.654823  0.109125  490.685272    4.382696  0.309407   
3388  583.913391    2.654274  0.108085  490.680237    4.372394  0.307441   
3389  583.911621    2.651568  0.107636  490.543884    4.369622  0.317528   

       right_ear                       
           cam_x      cam_y      prob  
0     742.039795  88.260811  0.022165  
1     744.182983  86.431641  0.192092  
2     743.937744  82.992607  0.200793  
3     745.902283  84.200920  0.219509  
4     746.644409  84.608818  0.270109  
...          ...        ...       ...  
3385  601.496216   3.505096  0.026705  
3386  601.506409   3.506500  0.026764  
3387  601.531128   3.490712  0.026583  
3388  601.496521   3.476666  0.026362  
3389  601.475342   3.472818  0.026377  

[3390 rows x 9 columns]